![i2b2 Logo](./images/transmart-logo.png)

# Using LLM for Clinical Note Analysis (Zero-shot)

This notebook demonstrates how to use a **locally hosted large language model (LLM)** with LangChain and Ollama to analyze clinical content in a zero-shot setting—without prior training or fine-tuning.

You will walk through a structured process for:
- Loading a local LLM (e.g., Qwen or DeepSeek via Ollama)
- Running basic clinical queries using system and user messages
- Creating reusable prompt templates for dynamic disease-specific explanations
- Preparing a prompt for structured clinical note extraction
- Applying the LLM to decode and analyze real patient notes

Each cell builds on the previous one, moving from simple interactions to more advanced, structured information extraction from unstructured clinical text.


In [65]:
# -----------------------------------------------------------
# 1. Load the Ollama Model
# -----------------------------------------------------------
# This cell loads a local Large Language Model (LLM) using LangChain's
# `ChatOllama` wrapper. Make sure the model has been pulled via Ollama CLI before use.

# To explore available models, visit:
# - Ollama Library: https://ollama.com/library
# - Hugging Face (for embeddings and additional models): https://huggingface.co/models

from langchain_ollama import ChatOllama

# Define the model name — make sure this model is already downloaded using:
#   ollama pull deepseek-v2:16b
model_name = "qwen2"  # Alternatives: "qwen2", "llama3", etc.

# Initialize the model with LangChain
model = ChatOllama(model=model_name)

print(f"✅ Model '{model_name}' is loaded and ready to use.")


✅ Model 'qwen2' is loaded and ready to use.


## 2. Basic Prompt Interaction - Simple Query

<img src="./images/basic_prompt.png" alt="i2b2 Logo" width="900">



In [66]:
# -----------------------------------------------------------
# 2. Run a Simple Clinical Query with System + User Prompts
# -----------------------------------------------------------
# This tests the model by asking a simple clinical question using structured messages.

from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import display, Markdown


messages = [
    SystemMessage(content=(
        "You are a knowledgeable medical provider. "
        "Provide clear, evidence-based explanations about a medical conditions."
    )),
    HumanMessage(content="What is asthma? What are its common symptoms and treatments?")
]

# Run inference
response = model.invoke(messages)

# Display result
display(Markdown("### Model Response:"))
display(Markdown(response.content))


### Model Response:

Asthma is a long-term (chronic) condition that inflames the airways in the lungs. This inflammation causes swelling and narrowing of these airways, leading to difficulty breathing. It affects millions of people worldwide.

### Common Symptoms of Asthma

Symptoms of asthma can vary greatly between individuals and can range from mild to severe:

1. **Coughing**: Often worse at night or early morning.
2. **Shortness of Breath**: Difficulty in breathing during physical activity, especially when running, climbing stairs, or exercising.
3. **Wheezing**: A whistling sound produced when you breathe, heard during both the intake and exhalation of air.
4. **Chest Tightness or Pressure**: Often described as a sensation of tightness in the chest that may make it feel difficult to open your chest cavity.
5. **Pain or Discomfort**: In some cases, especially in children, this can be confused with abdominal pain.

### Causes of Asthma

Asthma is often caused by exposure to triggers such as allergens (like pollen, dust mites, pet dander), cold air, exercise, respiratory infections, and irritants like smoke or chemical fumes. It's also influenced by genetics and factors including obesity, asthma history in the family, and viral infections.

### Treatments for Asthma

Asthma management is tailored to individual needs but typically includes:

1. **Medications**:
   - **Inhaled Corticosteroids**: These are medications used daily to reduce airway inflammation.
   - **Quick-Relief Inhalers (Rescue inhalers)**: Used during an asthma attack, they contain bronchodilators like albuterol or levalbuterol which help open the airways quickly.
   - **Long-term Control Medications**: These may include inhaled corticosteroids used daily to prevent symptoms and control inflammation. Oral medications, such as oral corticosteroids, are sometimes prescribed for severe asthma attacks.

2. **Avoiding Triggers**: Identifying and avoiding triggers is crucial. This might involve modifying the environment or adjusting lifestyle habits.

3. **Lifestyle Changes**:
   - Quitting smoking.
   - Maintaining good indoor air quality with regular cleaning, using air filters, and removing allergens like pet dander and dust mites.
   - Staying hydrated and well-rested can help manage symptoms.

4. **Education**: Understanding how to recognize asthma symptoms early and knowing what to do when an episode occurs is important for self-management.

5. **Monitoring and Adjusting Treatments**: Regular check-ups with a healthcare provider are crucial to adjust treatment plans based on changes in the condition of the patient or response to medications.

6. **Emergency Preparedness**: Having a written asthma action plan and quick-relief inhalers available at all times can help manage severe attacks effectively.

Asthma is managed through ongoing care, including regular check-ups with healthcare providers, monitoring symptoms, adjusting treatments as needed, and educating oneself on how to handle both mild and severe episodes. With proper management, most individuals with asthma are able to enjoy a good quality of life with only minimal impact from their condition.

## 3. Using ChatPromptTemplate for Dynamic Prompts

This section shows how to create reusable prompt templates using LangChain’s `ChatPromptTemplate`.

- **3.1**: Runs a dynamic medical query by filling in a condition (e.g., epilepsy) and a patient type (e.g., child).
- **3.2**: Prepares a prompt template for clinical note extraction. This will be used later in Section 4 when we insert actual patient notes.

Prompt templates make it easier to reuse the same structure with different inputs.

<img src="./images/prompt_template.png" alt="Prompt Template" width="900">


In [67]:
# -----------------------------------------------------------
# 3.1. Create a Reusable Prompt Template (Dynamic Querying)
# -----------------------------------------------------------
# This cell demonstrates how to build a dynamic prompt template using placeholders
# for different medical conditions and patient profiles. The prompt is populated
# with variables at runtime and sent to the LLM for inference.

from langchain.prompts import ChatPromptTemplate
from IPython.display import display, Markdown

# Define input variables
patient_type = "5 year-old"
disease = "epilepsy"

# Define a role-based prompt using variable placeholders
messages = [
    ("system",
     "You are a knowledgeable medical provider. Provide clear, evidence-based explanations appropriate for a {patient_type} patient."),

    ("human",
     "What is {disease}? What are its common symptoms and treatments?")
]

# Create a prompt template with dynamic inputs
prompt_template = ChatPromptTemplate.from_messages(messages)

# Fill the template with actual values
prompt = prompt_template.invoke({
    "patient_type": patient_type,
    "disease": disease
})

# Run the model with the constructed prompt
response = model.invoke(prompt)

# Display the generated answer
display(Markdown("### AI Response"))
display(Markdown(response.content))


### AI Response

Epilepsy is a condition where someone has seizures because their brain is not working as it should sometimes. Think of the brain like a traffic director that helps guide all your actions, from walking to playing games or eating. When something goes wrong in this system, like too many cars going too fast without following rules, it can cause strange things to happen.

### Common Symptoms
The most common symptom is having seizures, which look different for each person and can be:
1. **Staring spells** where you just stand still and don't do anything.
2. **Fits** that might make your arms or legs move in ways they normally don't.
3. **Confusion**, acting strange or not knowing what's going on around them.
4. **Convulsions** which are big shaking movements.

### Treatments
There are several ways to treat epilepsy, and the main goal is to help you have fewer seizures:
1. **Medicines**: Doctors might prescribe special pills that can calm down your brain signals, making seizures less likely.
2. **Special diets**: Sometimes following a very strict diet can help stop seizures in kids who respond well.
3. **Surgery**: If the problem comes from a specific spot in the brain, doctors might remove part of it to take away the bad signals causing seizures.
4. **Stimulation devices**: There are tiny devices that can be put into your body that help control seizure activity.

### Why Treatments Work
These treatments work by helping balance the signals in your brain. Just like how traffic lights help cars move safely, these treatments try to guide your brain's signals so they don't suddenly go crazy and cause seizures.

Remember, epilepsy is not something you can catch from someone else or eat too much of a certain thing; it's a bit more complex than that! It's just about the way your brain sometimes doesn't work quite right. Your doctors are here to help figure out what makes your brain act up so we can get those seizures under control.

If you ever have worries or questions, feel free to ask your parents or your healthcare provider to explain it in a way you understand better!

In [68]:
# -----------------------------------------------------------
# 3.2. Clinical Note Extraction Prompt
# -----------------------------------------------------------
# This cell defines a reusable prompt template to guide the LLM
# in extracting structured clinical information from a free-text note.

from langchain.prompts import ChatPromptTemplate

# Define the prompt structure using role-based messages
messages_notes = [
    ("system",
     "You are an advanced medical documentation assistant. "
     "Your task is to extract key clinical details from unstructured notes in a clear and structured format."),

    ("human",
     "Analyze the following clinical note:\n\n"
     "{patient_note}\n\n"
     "Extract the following:\n"
     "1. Patient demographics\n"
     "2. Chief complaints\n"
     "3. Current medications\n"
     "4. Does the patient have asthma? (Yes/No)\n\n"
     "Format your response using clear, structured bullet points.")
]

# Create the prompt template with a placeholder for the clinical note
prompt_template_notes = ChatPromptTemplate.from_messages(messages_notes)


## 4. Extract and Decode Notes from i2b2 Encounter Table

In this section, we explore how to work with real clinical notes stored in a simulated i2b2 `visit_dimension` table. These notes are encoded in BinHex format and need to be decoded before being used in a prompt.

### What You'll Do:
- **4.1**: Load and inspect the structure of the CSV file containing patient visits and encoded notes.
- **4.2**: Interactively select a patient and encounter to preview the decoded note.
- **4.3**: Use the previously defined prompt template to extract structured clinical information from a real decoded note.

This prepares the data for downstream tasks like summarization or diagnostic tagging using a large language model.


In [69]:
# -----------------------------------------------------------
# 4.1. Load and Explore Visit Data from i2b2-Mimicking CSV
# -----------------------------------------------------------
# This cell loads a simulated `visit_dimension` table similar to what is used in i2b2.
# It includes clinical notes stored in BinHex format and related visit metadata.

# Each record includes:
# - encounter_num: Unique visit ID
# - patient_num: Patient identifier
# - start_date, end_date: Visit timestamps
# - location_cd, location_path: Care setting info
# - visit_blob: BinHex-encoded clinical note

# Purpose:
# Load the data into a pandas DataFrame and inspect the structure before decoding.

import pandas as pd

# Load the visit-level dataset
csv_path = "datafiles/i2b2_encounter_table.csv"
df = pd.read_csv(csv_path)

# Preview the first 10 records
df.head(10)


,encounter_num,patient_num,start_date,end_date,inout_cd,location_cd,location_path,visit_blob
0,475303,1000000001,01/16/2003,01/16/2003,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
1,479681,1000000001,03/29/2007,03/29/2007,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
2,480315,1000000001,09/20/2007,09/20/2007,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
3,480903,1000000001,03/04/2008,03/04/2008,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x5468697320697320612032332D796561722D6F6C6420...
4,481398,1000000001,08/11/2008,08/11/2008,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x566973697420496E666F726D6174696F6E3A20202020...
5,482655,1000000001,05/18/2009,05/18/2009,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x566973697420496E666F726D6174696F6E3A0A202020...
6,471658,1000000002,04/17/1998,04/17/1998,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
7,472076,1000000002,01/04/1999,01/04/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
8,472473,1000000002,08/12/1999,08/12/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
9,472692,1000000002,12/01/1999,12/01/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...


In [70]:
# -----------------------------------------------------------
# 4.2. Interactive Exploration and Decoding of Clinical Notes
# -----------------------------------------------------------
# This cell allows users to interactively select a patient and encounter
# to preview the decoded clinical note (stored as BinHex in the dataset).

import binascii
import ipywidgets as widgets
from IPython.display import display, Markdown

# Step 1: Create dropdown for unique patient IDs
distinct_patients = sorted(df['patient_num'].unique())

patient_dropdown = widgets.Dropdown(
    options=distinct_patients,
    description="Patient #:",
    layout=widgets.Layout(width='25%')
)

# Step 2: Encounter dropdown (updates based on patient selection)
encounter_dropdown = widgets.Dropdown(
    options=[],
    description="Encounter #:",
    layout=widgets.Layout(width='25%')
)

# Step 3: Update encounters dynamically when patient is selected
def update_encounters(*args):
    patient_encounters = df[df['patient_num'] == patient_dropdown.value]['encounter_num'].tolist()
    encounter_dropdown.options = patient_encounters

# Step 4: Decode and display the selected clinical note
def decode_and_preview(patient_num, encounter_num):
    selected_row = df[
        (df['patient_num'] == patient_num) &
        (df['encounter_num'] == encounter_num)
    ].iloc[0]

    hex_blob = selected_row["visit_blob"].replace("0x", "")
    decoded_note = binascii.unhexlify(hex_blob).decode("utf-8", errors="ignore")

    display(Markdown(f"### Clinical Note Preview (Patient {patient_num}, Encounter {encounter_num})"))
    display(Markdown(f"```text\n{decoded_note}\n```"))

# Step 5: Set up dropdown linkage and render interface
patient_dropdown.observe(update_encounters, names='value')
update_encounters()

widgets.interact(
    decode_and_preview,
    patient_num=patient_dropdown,
    encounter_num=encounter_dropdown
)


interactive(children=(Dropdown(description='Patient #:', layout=Layout(width='25%'), options=(np.int64(1000000…

<function __main__.decode_and_preview(patient_num, encounter_num)>

In [71]:
# -----------------------------------------------------------
# 4.3. Analyze a Real Patient Note Using the Prompt
# -----------------------------------------------------------
# This cell selects a decoded clinical note for a given patient and encounter,
# fills the clinical extraction prompt (prepared in Cell 3.2), and runs it through the LLM.

# Define patient and encounter to analyze
patient_num = 1000000002
encounter_num = 475326

# Extract and decode the clinical note
selected_row = df[
    (df['patient_num'] == patient_num) &
    (df['encounter_num'] == encounter_num)
].iloc[0]

hex_blob = selected_row["visit_blob"].replace("0x", "")
decoded_note = binascii.unhexlify(hex_blob).decode("utf-8", errors="ignore")

# Fill the template with the note text
filled_prompt = prompt_template_notes.invoke({"patient_note": decoded_note})

# Invoke the model with the prompt
clinical_response = model.invoke(filled_prompt)

# Display the structured output
from IPython.display import display, Markdown
display(Markdown("### Extracted Clinical Information"))
display(Markdown(clinical_response.content))


### Extracted Clinical Information

### Extracted Information

**Patient Demographics:**
- **ID:** 1000000002
- **Date of Visit:** January 28, 2003
- **Encounter ID:** 475326
- **Age:** 37 years
- **Gender:** Female
- **Ethnicity:** White
- **Language Spoken:** Spanish

**Chief Complaints:**
1. Hair loss (persistent over several months)
2. Severe manic behavior episodes (intensifying in nature)
3. Headaches, specifically frontal region throbbing and only partially relieved by rest and analgesics
4. Ongoing respiratory difficulties including shortness of breath, wheezing, and difficulty swallowing

**Current Medications:**
- **Claritin Reditab 10mg:** Used for allergy relief with moderate symptomatic improvement in throat pain and respiratory discomfort.
- **Desipramine Hydrochloride 25mg:** Effective for depressive episodes but less so for manic episodes.

**Asthma Inquiry:**
- **Response:** No

### Additional Information
- **Tobacco Use:** Continuous, acknowledged as a factor contributing to her respiratory concerns.
- **Alcohol and Drug Use:** Refrained from alcohol and drugs.
- **Past Visit:** Last visit on September 24, 2001.

**Plan of Action:**
- **Referral:** Dermatology for alopecia evaluation.
- **Psychiatric Management:** Adjustment of bipolar disorder medication regimen (potentially increasing Desipramine or introducing a mood stabilizer).
- **Headache Treatment:** Trial of tricyclic antidepressant or prolonged analgesics, emphasizing hydration and rest.
- **Lifestyle Recommendations:** Smoking cessation and introduction of proton pump inhibitors for potential reflux management.

**Follow-Up:**
- Immediate contact if symptoms do not improve within the week.
- Monthly follow-up appointment to assess treatment effectiveness and make adjustments as needed.
- Importance of comprehensive education about chronic condition management highlighted.